In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.stats import norm

def calculate_max_current(f_name):
    df = pd.read_excel(f_name, engine='openpyxl')
    df = df[df.iloc[:, :1].values>0]  # 只考虑电压为正的情况
    df = df.iloc[: int(len(df)/2), :]  # 只考虑电压为正且递增的情况
    eps = 1e-12
    current_cols = df.iloc[:, 1:]
    # voltage_col = df.iloc[:, :1].values
    max_cur_lower_bound = current_cols.max(axis=0) * 0.95
    print(max_cur_lower_bound)
    
    
    
dir_name = './hardware_IV_and_condition_summary/hardware_data/'
for f_name in sorted(os.listdir(dir_name)):
    if not f_name.endswith('.xlsx'):
        continue
        
    print('\n'+f_name)
    calculate_max_current(dir_name + f_name)
    


I-V_data_0.7um_length_200nm_diameter_BT_4V_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_carbon_paste_no_wait.xlsx
current_cycle1     0.000254
current_cycle2     0.000207
current_cycle3     0.000161
current_cycle4     0.000030
current_cycle5     0.000049
current_cycle6     0.000048
current_cycle7     0.000134
current_cycle8     0.000172
current_cycle9     0.000057
current_cycle10    0.000129
current_cycle11    0.000107
dtype: float64

I-V_data_0.7um_length_200nm_diameter_BT_4V_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_probe_no_wait.xlsx
current_cycle1     0.002542
current_cycle2     0.002508
current_cycle3     0.002541
current_cycle4     0.002506
current_cycle5     0.002486
current_cycle6     0.002541
current_cycle7     0.002540
current_cycle8     0.002540
current_cycle9     0.002540
current_cycle10    0.002494
current_cycle11    0.002488
current_cycle12    0.002480
dtype: float64

I-V_data_0.7um_length_200nm_diameter_BT_4V_NA_third_etch_10min_Pb_ED_1h_

In [86]:
from scipy import interpolate
import matplotlib.pyplot as plt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pykalman import KalmanFilter

import torch
import torch.nn as nn
import torch.nn.functional as F

from scipy.stats import norm
# import matplotlib.pyplot as plt

def Kalman1D(observations, damping=0):
    # To return the smoothed time series data
    observation_covariance = damping
    initial_value_guess = observations[0]
    transition_matrix = 1
    transition_covariance = 0.1
    initial_value_guess
    kf = KalmanFilter(
            initial_state_mean=initial_value_guess,
            initial_state_covariance=observation_covariance,
            observation_covariance=observation_covariance,
            transition_covariance=transition_covariance,
            transition_matrices=transition_matrix
        )
    pred_state, state_cov = kf.smooth(observations)
    return pred_state

def LCIS(arr):
    """
    求最长连续[递增]*子序列
    返回最长连续递增子序列的
    ---------
    *该[递增]序列并不要求始终严格增，而是最多允许两次递减
    """
    decrease_cnt = 0
    start_index = 0
    end_index = 0
    sub_len = 0
    longest_start = 0
    longest_end = 0
    longest_len = 0
    decrease_point_0 = -1
    decrease_point_1 = -1
    for i in range(1, len(arr)):
        if arr[i] > arr[i-1]:
            end_index += 1
        else:
            decrease_cnt += 1
            if decrease_cnt == 1:
                end_index += 1
                decrease_point_0 = end_index
            elif decrease_cnt == 2:
                end_index += 1
                decrease_point_1 = end_index
            else:
                sub_len = end_index - start_index + 1
                if longest_len < sub_len:
                    longest_len = sub_len
                    longest_start = start_index
                    longest_end = end_index
                start_index = decrease_point_0
                decrease_point_0 = decrease_point_1
                decrease_point_1 = i
                end_index = i
                decrease_cnt = 2

    sub_len = end_index - start_index + 1
    if longest_len < sub_len:
        longest_len = sub_len
        longest_start = start_index
        longest_end = end_index

    return longest_start, longest_end


def mlesigma(f_name, st_indx, end_indx):
    """
    我们引入 MLE 方法来估计忆阻器电导值与对应电压水平下电导均值的比值的 mu 和 sigma
    函数返回 sigma 值，它的值反映了等效噪声水平的强度
    '''
    关于噪声分布形式设定与转换的说明：
    X ~ logNorm(a, b)  # m, v 是对数正态分布本身的均值和方差
    # 这说的是 X 是一个值永远为正的随机变量，只有取对数之后才是正态分布
    即 logX ~ Norm(mu, sigma)

    换算关系：
    m = exp(mu + sigma^2/2)  # 注意到 m 是 >0 的
    v = exp(2*mu + sigma^2) * exp(sigma^2 - 1)  

    因此，一个随机变量 Y 如果服从正态分布，即 Y ~ Norm(mu, sigma)
    就一定可以有一个 X = exp(Y), 使得 Y = logX ~ Norm(mu, sigma)

    所以，一个正态分布随机变量取指数得到的变量就服从对数正态分布

    具体到上面的代码，已知gaussian_kernel 服从正态分布，则 
            torch.exp(noise_sigma*gassian_kernel.sample(noise_sigma.size())) 
    服从对数正态分布，其中我们施加的 mu, sigma 都是对应的正态分布的参数，而非对数正态分布的参数

    又由于 (m, v) 和 (mu, sigma) 是一一映射，我们控制谁都一样，
    所以，为了方便和统一，今后：
    1）需要采样对数正态分布样本，都采取先用 mu, sigma 采样正态分布，再 exp 的方式
    2）需要估计对数正态分布参数，都采取先对对数正态分布样本取 log，再用 norm.fit 拟合的方式
    '''
    """
    eps = 1e-12
    data = pd.read_excel(f_name, engine='openpyxl').iloc[st_indx: end_indx+1, :]
    # data = data.iloc[st_indx: end_indx+1, :]
    
    conductance = data.drop(['voltage'], axis=1).div((data['voltage']+eps), axis=0)
    # conductance = conductance[(conductance!=np.inf).all(axis=1)]
    
    conductance_by_mean = np.array(conductance.div(conductance.mean(axis=1), axis=0)).reshape(1, -1)[0]
    samples = np.log(conductance_by_mean + eps)
    samples_fine = np.delete(conductance_by_mean, np.isfinite(samples)==False)

    mu_hat, sigma_hat = norm.fit(samples_fine)  # 通过极大似然估计得到 sigma\hat，不固定均值为 0
    # mu_hat, sigma_hat = norm.fit(samples_fine, floc=0)  # 通过极大似然估计得到 sigma\hat，固定均值为 0
    return mu_hat, sigma_hat


def calculate_smoothed_cmean(f_name):
    df = pd.read_excel(f_name, engine='openpyxl')
    eps = 1e-12
    current_cols = df.iloc[:, 1:]
    voltage_col = df.iloc[:, :1].values

    conductance_cols = current_cols.div(voltage_col + eps)
    conductance_mean_col = conductance_cols.mean(axis=1).values
    conductance_mean_col_head100 = conductance_mean_col[1: 101]
    c_mean_smooth = Kalman1D(conductance_mean_col_head100)
    # c_mean_smooth = c_mean_smooth.T[0]
    return c_mean_smooth

In [75]:
np.array([[1,2,np.nan],[1,2,np.nan]])

array([[ 1.,  2., nan],
       [ 1.,  2., nan]])

In [74]:
np.nanmean(np.array([[1,2,np.nan],[1,2,np.nan]]),axis=1)

array([1.5, 1.5])

In [114]:
eps = 1e-8
dir_name = './hardware_IV_and_condition_summary/hardware_data/'

file2mu = {}
file2sigma = {}
file2usability = {}

for f_name in sorted(os.listdir(dir_name)):
    
    df = pd.read_excel(dir_name+f_name, engine='openpyxl')
    
    
    df = df[df.iloc[:, :1].values>0]  # 只考虑电压为正的情况
    df = df.iloc[: int(len(df)/2), :]  # 只考虑电压为正且递增的情况

    max_cur_lower_bound = df.max(axis=0) * 0.99
    df = df[df.iloc[:, :] < max_cur_lower_bound]

    current_cols = df.iloc[:, 1:]
    voltage_col = df.iloc[:, :1].values
    conductance = current_cols / (voltage_col + eps)
    
    c_mean = conductance.mean(axis=1)
    c_std = conductance.std(axis=1)
    # c_mean = Kalman1D(np.array(c_mean).reshape(-1, 1), damping=1)
    
    longest_start, longest_end = LCIS(np.array(c_mean))
    # print(longest_start, longest_end)
    
    conductance_by_mean = conductance.div(c_mean, axis=0)[longest_start: longest_end+1]
    conductance_by_mean_array = np.array(conductance_by_mean).reshape(1, -1)[0]
    conductance_by_mean_array_dropna = conductance_by_mean_array[~np.isnan(conductance_by_mean_array)]
    
    samples = np.log(conductance_by_mean_array_dropna + eps)
    samples_dropinf = samples[~np.isinf(samples)]
    samples_fine = sorted(samples_dropinf)[int(len(samples_dropinf)*0.05): int(len(samples_dropinf)*0.95)+1]

    mu_hat, sigma_hat = norm.fit(samples_fine)  # 通过极大似然估计得到 sigma\hat，不固定均值为 0
    # mu_hat, sigma_hat = norm.fit(samples_fine, floc=0)  # 通过极大似然估计得到 sigma\hat，固定均值为 0

    # print(mu_hat, sigma_hat)
    usability = (longest_end - longest_start) / 100. * (1 - c_std.sum() / c_mean.sum())
    # usability = (longest_end - longest_start) / 100. * (1 - sigma_hat / mu_hat)
    # usability = (longest_end - longest_start + 1) / int(len(df) / 2)
    
    file2mu[f_name] = mu_hat
    file2sigma[f_name] = sigma_hat
    file2usability[f_name] = usability
    # print(usability, sigma_hat)

file2mu, file2sigma, file2usability

({'I-V_data_0.7um_length_200nm_diameter_BT_4V_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_carbon_paste_no_wait.xlsx': -1.3784557844565033,
  'I-V_data_0.7um_length_200nm_diameter_BT_4V_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_probe_no_wait.xlsx': -0.2401378082577507,
  'I-V_data_0.7um_length_200nm_diameter_BT_4V_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_silver_paste_no_wait.xlsx': -1.760113237657738,
  'I-V_data_0.7um_length_200nm_diameter_BT_4V_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_5V_silver_paste_no_wait.xlsx': -0.14931369516006454,
  'I-V_data_0.7um_length_200nm_diameter_BT_4V_NA_third_etch_25min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_8V_carbon_paste_no_wait.xlsx': -0.5641224345339781,
  'I-V_data_1.2um_length_200nm_diameter_BT_4V_NA_third_etch_15min_Pb_ED_10h_180+210C_FAI_PMMA_200nm_Ag_memory_6V_probe_no_wait.xlsx': -0.5390298155671931,
  'I-V_data_1.2um_length_200nm_diameter_BT_4V_NA_third_etch_15min_Pb_ED_10h_1

In [118]:
np.save('file2sigma.npy', file2sigma)
np.save('file2usability.npy', file2usability)